# Import package

In [1]:
import threading
from queue import Queue

from bs4 import BeautifulSoup
import pandas as pd
import requests

# Get Page Content

In [2]:
def get_page(url):
    content = requests.get(url).text    
    soup = BeautifulSoup(content, "html.parser")
    
    return soup

# Crawl the page by giving url

In [3]:
def crawler_job(url, q):
    US_exchange_rate_data = {}
    try:
        annual_page_content = get_page(url)
        row_data = annual_page_content.find(class_='text12').find_all('tr')[1:]
        for row in row_data:
            date, NTD_USD = [r.text for r in row.find_all('td')]
            US_exchange_rate_data[date] = NTD_USD
    except AttributeError:
        print(url)
        print('Error')

    q.put(US_exchange_rate_data)
    print(url, 'finish')

# Start Crawl

In [4]:
BASE_URL = 'https://www.cbc.gov.tw/'
US_exchange_rate_data = dict()

page_content = get_page('https://www.cbc.gov.tw/lp.asp?ctNode=382&CtUnit=128&BaseDSD=7')
annual_page_link = page_content.find(class_='list').find_all('a')
annual_page_link = ['{}{}'.format(BASE_URL, annual_link.get('href')) for annual_link in annual_page_link]

threads = []
q = Queue()
for url in annual_page_link:
    t = threading.Thread(target=crawler_job, args=(url, q))
    t.start()
    threads.append(t)

for thread in threads:
    thread.join()

print('Done')
for _ in range(len(annual_page_link)):
    US_exchange_rate_data.update(q.get())

https://www.cbc.gov.tw/ct.asp?xItem=2284&ctNode=382&mp=1
Error
https://www.cbc.gov.tw/ct.asp?xItem=2284&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=63271&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=37995&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=33461&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=79504&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=41677&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=2287&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=37993&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=43548&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=39919&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=58589&ctNode=382&mp=1https://www.cbc.gov.tw/ct.asp?xItem=52733&ctNode=382&mp=1 finish
 finish
https://www.cbc.gov.tw/ct.asp?xItem=45403&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?xItem=2285&ctNode=382&mp=1 finish
https://www.cbc.gov.tw/ct.asp?

# Output CSV

In [5]:
pd.DataFrame.from_dict(US_exchange_rate_data, orient='index').to_csv('data.csv')

TypeError: 'bool' object is not iterable